# Modelling

The main objectives for this notebook are:
* To develop a model that will satisfiy our modelling objective
* To properly evaluate the developed model
* To have a trained model ready for deployment


## good things to do
1. Have a proper baseline
2. Perform post-modellign steps - threshold selection, explainability, false positives / false negatives
3. Use MLFlow for experiment tracking
4. Build an ML training pipeline using Kedro/ZenML/Metaflow/etc


In [1]:
import os
import sys
import warnings

import joblib
import mlflow
import numpy as np
import plotly.express as px
import polars as pl
import shap
from optuna.integration.mlflow import MLflowCallback
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import average_precision_score, roc_auc_score
from sklearn.model_selection import cross_val_score, train_test_split

c:\Users\larib\Projects\Api security\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
